## Task 2: relevance

Some additional functions...

In [ ]:
def checkConvert(line, n):
    '''check if the line has exactly n objects and convert it to a list of integers'''
    if len(line) != n: #O(1)
        # print("Error: incorrect number of parameters.")
        raise RuntimeError("incorrect number of parameters.")
    else:
        line = list(map(int, line)) #O(n)
        return line
    
# Time complexity: O(n)
# Space complexity: O(n)

In [ ]:
def calcRelevance(F, a):
    ''' calculate relevance for each object
        F - matrix of the objects' features 
        a - list of parameters' values '''
    R = [] # here we store relevances for objects 
    for i in range(len(F)): #O(d)
        relevance = 0
        for j in range(len(a)): #O(n)
            relevance += a[j] * F[i][j]
        R.append(relevance)
    return R 
# Time complexity: O(d⋅n)
# Space complexity: O(d)

In [ ]:
def changeValue(F, i, j, v):
    ''' F - matrix of the objects' features 
        i - index of the object in F
        j - index of the feature in i-th obgect in F
        v - new value for jth feature'''
    
    F[i][j] = v
    return F 
# Time complexity: O(1)
# Space complexity: O(1)

The code for this task. First try  

For the first try whole time complexity is $O(d \cdot n + q \cdot d \cdot n + q\cdot d \cdot \log d)$  
and for space is $O(d \cdot n)$ to store F matrix


In [ ]:
n = int(input()) # number of the parameters 

line2 = input().split()
a = checkConvert(line2, n) # list of the parameters with check for appropriate amount

d = int(input()) #amount of the objects

F = [] # features of d objects 
for line in range(d):
    line = checkConvert(input().split(), n) #O(d⋅n), F - O(d*n)
    F.append(line)

q = int(input()) # amount of requests

for line in range(q):
    line = list(map(int, input().split()))
    if line[0] == 1: # Query type 1: Output top-k most relevant objects
        R = calcRelevance(F, a) #O(d⋅n)
        indexed_list = list(enumerate(R))
        sorted_list = sorted(indexed_list, key=lambda item: item[1], reverse=True) #O(dlogd)
        indices = [item[0] + 1 for item in sorted_list] # +1 for 1-based indexing
        print(*indices[0:line[1]])
        #for this if-block Time complexity: O(d⋅n + dlogd), space: O(d)
    elif line[0] == 2: # Query type 2: Change value in matrix F 
        F = changeValue(F, line[1]-1, line[2]-1, line[3]) #O(1) # -1 for 0-based indexing
    else:
        print("Wrong request!")
# for q request in the worst case time complexity is O(q⋅d⋅n + q⋅dlogd)


1 10
1 10 9 8 7 6 5 4 3 2
4 1 10 9 8 7 6 5 3 2


The code can be improved. For now calcRelevance() is called every time we get type 1 request but in fact type1 almost every time needs just sorting. Additionaly when we have type 2 request we can change matrix F only locally for that object. Also heapsort will a little bit faster. I'll implement those ideas and also performe syntaxis optomization 

In [ ]:
import heapq


def calcRelevance2(F: list[list], a: list) -> list: #t O(d⋅n), s O(d)
    """Calculate relevances for objects using list comprehension"""
    return [sum(a[j] * F[i][j] for j in range(len(a))) for i in range(len(F))] 


def sortTopRelevance(R, k): # t O(d + klogk), s O(k) for top k and O(d) for R
    """Return indices of the top-k most relevant objects."""
    # Using heap for faster top-k extraction
    top_k = heapq.nlargest(k, enumerate(R), key=lambda item: item[1])
    return [item[0] + 1 for item in top_k]  # +1 for 1-based indexing


# Main program
number_of_parameters = int(input())
a = checkConvert(input().split(), n)  # list of parameters with check for appropriate amount #O(n)
d = int(input())  # number of objects

F = []  # features of d objects
for _ in range(d):
    F.append([i for i in input().split()])

R = calcRelevance2(F, a) # Calculate initial relevance 
# сюда вставить соортировку 
q = int(input())  # number of requests

for _ in range(q):
    line = list(map(int, input().split()))
    if line[0] == 1: # Query type 1: Output top-k most relevant objects, t O(q⋅(d+klogk))
        k = line[1]
        indices = sortTopRelevance(R, k) # это убрать и выводить только k элементов 
        print(*indices)
    elif line[0] == 2: # Query type 2: Change value in matrix F and update relevance, t O(q⋅n)
        i, j, v = line[1] - 1, line[2] - 1, line[3]  # Convert to 0-based index
        R[i] = R[i] - F[i][j] + v
        F[i][j] = v 
    else:
        print("Wrong request!")

1 10
1 10 9 8 7 6 5 4 3 2
4 1 10 9 8 7 6 5 3 2


Total time complexity: $O(q⋅(d+klogk)+q⋅n)$, space complexity $O(d⋅n)$.